In [1]:
import pandas as pd
import numpy as np

df = pd.read_excel("data/data.xlsx")

df.head()

,age,income,student,credit_rating,buys_computer
0,< = 30,high,no,fair,no
1,< = 30,high,no,excellent,no
2,31 ... 40,high,no,fair,yes
3,> 40,medium,no,fair,yes
4,> 40,low,yes,fair,yes


In [16]:
# A1
# Calculate prior probability

print(f"There are {df['buys_computer'].nunique()} classes for this problem.")

pos = "yes"
neg = "no"

# Total number of instances
total = df.shape[0]

# Number of positive instances and negative instances
num_NO = len(df[df['buys_computer'] == pos])
num_YES = len(df[df['buys_computer'] == neg])

# Calculate probability 
print(f"Probability of P({pos}) is: {num_YES / total}")
print(f"Probability of P({neg}) is: {num_NO / total}")

There are 2 classes for this problem.
Probability of P(yes) is: 0.35714285714285715
Probability of P(no) is: 0.6428571428571429
